In [52]:
import argparse
import os
import cv2
import sys
import pickle
import random
import numpy as np
from matplotlib import pyplot as plt

In [53]:
### reads the image from given location
def read_img(img):
    img = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
    return img

### Returns the ssd for a given pair of descriptors
def get_ssd(d1,d2):
    length = len(d1)
    sum = 0
    d3 = d1 - d2
    sum = sum + np.linalg.norm(d3)
    return sum

### Returns matched keypoints
def get_mapped_keypoints(kp1,kp2,des1,des2):
    sd = 0
    mp1 = []
    mp2 = []
    s = []
    len1 = len(des1)
    len2 = len(des2)
    j_val = 0
    for i in range(0,len1):
        min1 = sys.maxsize
        min2 = sys.maxsize
        for j in range(0,len2):
            sd = get_ssd(des1[i],des2[j])
            s.append(sd)
            if(sd < min1):
                min2 = min1
                min1 = sd
                j_val = j
            elif(sd<min2):
                min2 = sd
        ratio = min1/min2
        if(ratio < 0.7):
            print("Ratio:" + str(ratio))
            print("Matched keypoints for"+ str(i) +", "+str(j_val))
            mp1.append(kp1[i].pt)
            mp2.append(kp2[j_val].pt)
            print(kp1[i].pt,kp2[j_val].pt)
        #raise NotImplementedError
    return mp1,mp2

### detect features using SIFT
def detect_sift(img1,img2):
    sift = cv2.xfeatures2d.SIFT_create()
    kp1,des1 = sift.detectAndCompute(img1,None)
    kp2,des2 = sift.detectAndCompute(img2,None)
    kimg1 = cv2.drawKeypoints(img1,kp1,None)
    kimg2 = cv2.drawKeypoints(img2,kp2,None)
    cv2.imwrite("./results/sift_nevada5.jpg",kimg1)
    cv2.imwrite("./results/sift_nevada4.jpg",kimg2)
    #print(kp1[0].pt)
    print("Matching key points wait for 1-2 minutes")
    mp1,mp2 = get_mapped_keypoints(kp1,kp2,des1,des2)
    print(len(mp1))
    print(len(mp2))
    #print(des1[0])
    return mp1,mp2

### Returns the homography matrix
def homography_func(ri,mp1,mp2):
    H = []
    A = []
    src =[]
    dest = []
    print(ri)
    for i in ri:
        src.append(list(mp1[i]))
        dest.append(list(mp2[i]))
        x1 = mp1[i][0]
        y1 = mp1[i][1]
        x2 = mp2[i][0]
        y2 = mp2[i][1]
        x2x1 = -x2*x1
        x2y1 = -x2*y1
        y2x1 = -y2*x1
        y2y1 = -y2*y1
        A.append([x1,y1,1,0,0,0,-x2x1,-x2y1,-x2])
        A.append([0,0,0,x1,y1,1,-y2x1,-y2y1,-y2])
    A = np.matrix(A)
    u,s,vh = np.linalg.svd(A)
    #vh = vh/vh[8]
    #print(vh)
    minv = vh[8].tolist()
    minv = list(minv[0])
    #print(minv)
    div = minv[8]
    for i in range(0,len(minv)):
        minv[i] = minv[i] / div
    i = 0
    H = []
    h_elem = []
    while(i<len(minv)):
        for j in range(i,i+3):
            if(j >=6 and j<=7):
                h_elem.append(-minv[j])
            else:
                h_elem.append(minv[j])
        H.append(h_elem)
        h_elem = []
        i = i+3
    print("Computed homography matrix: ")
    print(H)
    
    #src = np.array(src)
    #dest = np.array(dest)
    #h, status = cv2.findHomography(src, dest)
    #print("Homography matrix from cv2: ")
    #print(h)
    '''
    elem1 = list(mp1[0])
    elem2 = list(mp2[0])
    elem1.append(1)
    print("Keypoint from img 1")
    print(elem1)
    print("Keypoint from img2")
    print(elem2)
    print("Computed keypoint")
    kp1 = np.matmul(H,elem1)
    kp2 = np.matmul(h,elem1)
    
    print(kp1[0]/kp1[2])
    print(kp1[1]/kp1[2])
    print(kp2[0]/kp2[2])
    print(kp2[1]/kp2[2])
    '''
    return H

### Computes the mean squared error for points
def find_error(p1,p2,H):
    p1 = list(p1)
    p2 = list(p2)
    H  = list(H)
    #print(p1)
    #print(H)
    p1.append(1)
    #print(p1)
    #print(np.shape(p1))
    #print(np.shape(H))
    np2 = np.matmul(H,p1)
    #nx1 = np2[0]/np2[2]
    #ny1 = np2[1]/np2[2]
    np2 = np2/np2[2]
    p2.append(1)
    #print("Computed keypoint: ")
    #print(np2)
    #print("Original keypoint: ")
    #print(p2)
    err = np.square(np.subtract(p2, np2)).mean()
    return err
    
### Runs ransac and return Homography matrix and inliers    
def ransac_func(mp1,mp2):
    num_to_select = 4
    index = []
    inliers = []
    for i in range(0,len(mp1)):
        index.append(i)
    ri = random.sample(index, num_to_select)
    ri.sort()
    #print(ri)
    H = homography_func(ri,mp1,mp2)
    best_inliers = ri
    best_h = []
    iterations = 1000
    
    for j in range(0,iterations):
        print("iteration: "+str(j))
        for i in range(0,len(mp1)):
            error = find_error(mp1[i],mp2[i],H)
            #print(error)
            if(error < 1):
                inliers.append(i)
        print("inliers: ")
        print(inliers)
        print("best inliers: ")
        print(best_inliers)
        if(len(inliers) >= len(best_inliers)):
            best_inliers = inliers
            best_h = H
        if(len(best_inliers) > 0.6*len(mp1)):
            break
        bi = random.sample(index,num_to_select)
        bi.sort()
        print("new random indices: ")
        H = homography_func(bi,mp1,mp2)
        inliers = []
    print("end")
    return best_h,best_inliers
    
def stitch_func(img1,img2,H):
    s1 = np.shape(img1)
    s2 = np.shape(img2)
    print(s1[0],s1[1])
    print(s2[0],s2[1])
    H = np.array(H)
    corners1 = np.float32([[0,0],[s1[1],0],[0,s1[0]],[s1[1],s1[0]]])
    corners2 = np.float32([[0,0],[s2[1],0],[0,s2[0]],[s2[1],s2[0]]])
    corners1 = corners1.reshape(-1,1,2)
    corners2 = corners2.reshape(-1,1,2)
    warp_corners1 = cv2.perspectiveTransform(corners1, H)
    corners = np.concatenate((warp_corners1,corners2),axis=0)
    (min_x,min_y) = np.int32(corners.min(axis=0).reshape(-1))
    (max_x,max_y) = np.int32(corners.max(axis=0).reshape(-1))
    tx,ty = -min_x,-min_y
    tran_matrix = np.array([[1,0,tx],[0,1,ty],[0,0,1]])
    tran_H = tran_matrix.dot(H)
    
    result = cv2.warpPerspective(img1,tran_H,(max_x-min_x,max_y-min_y))
    result[ty:s2[0]+ty,tx:s2[1]+tx] = img2
    #cv2.imwrite("./results/resultant.jpg",result)
    return result
    #print(np.shape(result))
    
    

In [54]:
img1 = read_img("./data/nevada5.jpg")
img2 = read_img("./data/nevada4.jpg")
img3 = read_img("./data/nevada3.jpg")
print(np.shape(img1))
print(np.shape(img2))
#raise NotImplementedError
mp1,mp2 = detect_sift(img1,img2)

(2592, 3888)
(2592, 3888)
Matching key points wait for 1-2 minutes
Ratio:0.5097040383212538
Matched keypoints for6, 293
(66.60956573486328, 1466.54443359375) (1466.4959716796875, 1441.8406982421875)
Ratio:0.4764983826790785
Matched keypoints for12, 302
(107.03238677978516, 1473.087890625) (1494.462646484375, 1446.903564453125)
Ratio:0.4173517448174388
Matched keypoints for13, 297
(117.88528442382812, 334.92669677734375) (1476.9990234375, 423.4190979003906)
Ratio:0.6084356353056798
Matched keypoints for15, 309
(119.56205749511719, 1473.822998046875) (1504.87255859375, 1447.5272216796875)
Ratio:0.36472757426996866
Matched keypoints for16, 310
(121.38249206542969, 1449.6907958984375) (1505.3612060546875, 1425.671875)
Ratio:0.3048151778722766
Matched keypoints for18, 308
(150.39938354492188, 361.1566467285156) (1504.2117919921875, 442.4951171875)
Ratio:0.3380814500682852
Matched keypoints for20, 321
(159.2357177734375, 1475.2857666015625) (1537.77587890625, 1448.8311767578125)
Ratio:0.4577

Ratio:0.200440996806886
Matched keypoints for149, 511
(627.94482421875, 1463.2857666015625) (1945.6273193359375, 1435.5126953125)
Ratio:0.542816569361844
Matched keypoints for150, 498
(627.9751586914062, 480.07049560546875) (1916.8988037109375, 505.3311462402344)
Ratio:0.2598894365752158
Matched keypoints for151, 502
(637.2669677734375, 473.5498962402344) (1925.558837890625, 500.2223205566406)
Ratio:0.30488137813265626
Matched keypoints for154, 510
(657.1776733398438, 461.046630859375) (1943.25341796875, 486.72613525390625)
Ratio:0.5889918730778073
Matched keypoints for157, 516
(685.3526000976562, 510.39508056640625) (1971.138671875, 530.6117553710938)
Ratio:0.28892013048975684
Matched keypoints for159, 521
(712.5227661132812, 599.6898803710938) (1999.1644287109375, 613.3953857421875)
Ratio:0.1758616436692624
Matched keypoints for160, 522
(715.4979248046875, 584.1703491210938) (2002.3060302734375, 598.986083984375)
Ratio:0.32407807017772305
Matched keypoints for166, 537
(736.0091552734

Ratio:0.6510398710724822
Matched keypoints for283, 660
(967.9524536132812, 2037.7052001953125) (2301.73046875, 1993.5006103515625)
Ratio:0.6645328167888472
Matched keypoints for284, 634
(968.1255493164062, 1486.6920166015625) (2265.06640625, 1456.1568603515625)
Ratio:0.5156908851026916
Matched keypoints for285, 623
(968.8528442382812, 820.8313598632812) (2250.335693359375, 810.2765502929688)
Ratio:0.3569935513287778
Matched keypoints for288, 643
(970.8168334960938, 1555.8223876953125) (2274.51708984375, 1523.74560546875)
Ratio:0.48624634117153237
Matched keypoints for289, 665
(971.9724731445312, 2111.04248046875) (2308.548828125, 2064.95556640625)
Ratio:0.3216395508418208
Matched keypoints for290, 666
(971.9724731445312, 2111.04248046875) (2308.548828125, 2064.95556640625)
Ratio:0.520268917350348
Matched keypoints for292, 669
(976.9259643554688, 2296.23193359375) (2314.01025390625, 2244.964111328125)
Ratio:0.38800778881922315
Matched keypoints for294, 670
(978.364501953125, 2141.526123

Ratio:0.667923876871393
Matched keypoints for464, 819
(1250.134521484375, 2179.92822265625) (2575.409423828125, 2144.9423828125)
Ratio:0.5844057250249671
Matched keypoints for465, 809
(1250.2462158203125, 1445.3109130859375) (2548.228515625, 1413.1192626953125)
Ratio:0.1627816431628982
Matched keypoints for471, 811
(1260.23876953125, 1427.8492431640625) (2557.749267578125, 1395.7764892578125)
Ratio:0.661158847267954
Matched keypoints for474, 827
(1264.1312255859375, 2195.767822265625) (2590.4638671875, 2162.724853515625)
Ratio:0.6966096721444548
Matched keypoints for477, 807
(1269.0130615234375, 1422.7493896484375) (2542.41552734375, 1392.8321533203125)
Ratio:0.28541403771780505
Matched keypoints for478, 799
(1269.2110595703125, 489.53961181640625) (2527.279052734375, 460.6340637207031)
Ratio:0.21173507827040774
Matched keypoints for481, 836
(1274.248046875, 2284.400146484375) (2602.88037109375, 2252.135986328125)
Ratio:0.599693859593336
Matched keypoints for482, 826
(1276.517456054687

Ratio:0.601065241207317
Matched keypoints for710, 988
(1637.2674560546875, 1835.3223876953125) (2958.818359375, 1811.15283203125)
Ratio:0.6828593557435816
Matched keypoints for712, 840
(1638.302978515625, 1345.3883056640625) (2611.67578125, 1422.4459228515625)
Ratio:0.6839730317113628
Matched keypoints for715, 1109
(1644.910888671875, 1340.1031494140625) (3096.34228515625, 1050.3206787109375)
Ratio:0.46296970916460634
Matched keypoints for716, 982
(1645.8031005859375, 1335.4130859375) (2949.86474609375, 1295.853759765625)
Ratio:0.4948646748515624
Matched keypoints for719, 993
(1647.926025390625, 2148.064453125) (2972.309326171875, 2134.6435546875)
Ratio:0.6166427275906662
Matched keypoints for721, 995
(1650.2305908203125, 1938.6661376953125) (2973.53466796875, 1918.1583251953125)
Ratio:0.3750878163838672
Matched keypoints for722, 957
(1651.1915283203125, 487.17413330078125) (2914.27294921875, 423.12677001953125)
Ratio:0.6258096075851074
Matched keypoints for728, 961
(1660.343994140625,

Ratio:0.645338749690169
Matched keypoints for900, 1159
(1892.5828857421875, 1124.501708984375) (3206.776123046875, 1070.125)
Ratio:0.4454563048841341
Matched keypoints for901, 1162
(1893.2618408203125, 1249.710205078125) (3211.275146484375, 1200.8377685546875)
Ratio:0.1926865405148035
Matched keypoints for908, 1163
(1895.9327392578125, 1259.5966796875) (3213.960693359375, 1211.5491943359375)
Ratio:0.5211222845990894
Matched keypoints for914, 1174
(1904.2218017578125, 1558.9725341796875) (3241.27294921875, 1527.0380859375)
Ratio:0.4496972021683745
Matched keypoints for920, 1175
(1906.027587890625, 1657.78173828125) (3244.557861328125, 1631.2572021484375)
Ratio:0.2592020968812704
Matched keypoints for922, 1164
(1907.3270263671875, 1130.08203125) (3222.391845703125, 1074.7197265625)
Ratio:0.4946791747490041
Matched keypoints for940, 1181
(1920.75439453125, 1610.6566162109375) (3259.9462890625, 1581.6064453125)
Ratio:0.4023815733342953
Matched keypoints for941, 1172
(1920.8668212890625, 11

Ratio:0.6769369495346497
Matched keypoints for1133, 1371
(2128.63037109375, 2346.5712890625) (3516.08349609375, 2384.04248046875)
Ratio:0.6155178460344478
Matched keypoints for1136, 1365
(2129.36474609375, 2133.59130859375) (3506.48974609375, 2150.10400390625)
Ratio:0.36853053028067867
Matched keypoints for1140, 1366
(2132.08349609375, 1931.370849609375) (3506.560546875, 1930.4886474609375)
Ratio:0.23814058743282188
Matched keypoints for1143, 1345
(2132.9091796875, 1153.6978759765625) (3476.61083984375, 1091.2611083984375)
Ratio:0.5849759431166285
Matched keypoints for1145, 1368
(2134.271240234375, 2069.34375) (3511.200927734375, 2080.621826171875)
Ratio:0.5451237086010512
Matched keypoints for1147, 1369
(2134.97021484375, 2061.090576171875) (3512.054931640625, 2071.634765625)
Ratio:0.3692629438769643
Matched keypoints for1149, 1348
(2135.9697265625, 1145.25341796875) (3479.281005859375, 1082.1278076171875)
Ratio:0.2504233780356625
Matched keypoints for1150, 1351
(2136.256591796875, 11

Ratio:0.660275527735553
Matched keypoints for1863, 1141
(3564.603271484375, 737.0602416992188) (3179.696533203125, 1229.9522705078125)
Ratio:0.683656529635198
Matched keypoints for1891, 1462
(3647.197021484375, 679.1508178710938) (3744.874267578125, 1111.060302734375)
Ratio:0.5836930481255506
Matched keypoints for1907, 278
(3740.705078125, 641.4277954101562) (1429.5599365234375, 394.885498046875)
Ratio:0.524507851265811
Matched keypoints for1941, 1122
(3813.611083984375, 599.8556518554688) (3138.658935546875, 1037.7867431640625)
Ratio:0.6552466133424486
Matched keypoints for1942, 1325
(3815.838623046875, 114.21875762939453) (3455.728759765625, 1137.9134521484375)
Ratio:0.6661649191386538
Matched keypoints for1960, 1106
(3844.603515625, 593.9695434570312) (3092.515625, 1272.0728759765625)
460
460


In [55]:
print(mp1[0])
print(mp2[0])

(66.60956573486328, 1466.54443359375)
(1466.4959716796875, 1441.8406982421875)


In [9]:
#img1 = read_img("./data/nevada5.jpg")
#img2 = read_img("./data/nevada4.jpg")

In [10]:
'''
for i in range(0,len(mp2)):
    mp2[i] = list(mp2[i])
    mp2[i][0] = mp2[i][0] + 3888
    mp2[i] = tuple(mp2[i])
'''

In [11]:
#print(mp2[2])

(5393.3612060546875, 1425.671875)


In [12]:
'''
vis = np.concatenate((img1, img2), axis=1)
for i in range(50,70):
    cv2.line(vis,(int(mp1[i][0]),int(mp1[i][1])),(int(mp2[i][0]),int(mp2[i][1])),(255,0,0),5)
'''

In [ ]:
#cv2.namedWindow("combine",cv2.WINDOW_NORMAL)
#cv2.imshow("combine",vis)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [56]:
H,inliers = ransac_func(mp1,mp2)
#inliers.sort()
print("final inliers")
print(inliers)


'''
for i in range(0,len(mp1)):
    error = find_error(mp1[i],mp2[i],H)
    print(error)
'''

[3, 286, 386, 401]
Computed homography matrix: 
[[0.6938451744389694, 0.03551566281841538, 1353.7955286867812], [-0.12105670477089833, 0.8915666098858296, 132.8151185537772], [-8.05776838396246e-05, -5.760722420309456e-07, 1.0]]
iteration: 0
inliers: 
[1, 3, 4, 6, 8, 11, 14, 16, 19, 23, 24, 27, 29, 39, 264, 278, 286, 309, 343, 344, 348, 349, 351, 355, 356, 358, 386, 387, 388, 401]
best inliers: 
[3, 286, 386, 401]
new random indices: 
[124, 366, 373, 439]
Computed homography matrix: 
[[-2.1447433101100675, -1.4336208878677097, 4188.45741511], [-0.6497934880004688, -0.9637184262269658, 1833.5504881412435], [-0.000516467131012003, -0.0004262415743790172, 1.0]]
iteration: 1
inliers: 
[124, 366, 373, 439]
best inliers: 
[1, 3, 4, 6, 8, 11, 14, 16, 19, 23, 24, 27, 29, 39, 264, 278, 286, 309, 343, 344, 348, 349, 351, 355, 356, 358, 386, 387, 388, 401]
new random indices: 
[86, 263, 291, 456]
Computed homography matrix: 
[[-0.6567076680560888, -0.8615394594132494, 2704.779263096289], [-0.1297

inliers: 
[111, 115, 116, 120, 122, 133, 134, 139, 140, 141, 142, 143, 144, 145, 146, 161, 164, 167, 182, 184, 246, 250, 251, 312, 334, 336]
best inliers: 
[28, 44, 50, 55, 56, 57, 66, 67, 68, 69, 76, 82, 83, 85, 87, 98, 113, 121, 152, 158, 180, 183, 189, 191, 199, 204, 212, 216, 218, 221, 222, 223, 224, 226, 227, 228, 229, 231, 268, 283, 296, 297, 304, 307, 317, 363, 364, 365, 366, 367, 369, 374, 376, 378, 381, 382, 383, 384, 392, 395, 396, 424, 425, 426, 427, 430, 432]
new random indices: 
[0, 76, 301, 376]
Computed homography matrix: 
[[0.6951199422920683, 0.00048780865425010104, 1378.6271766791454], [-0.11173110635212792, 0.8721528441326112, 130.06721045978182], [-7.450088340963736e-05, -1.56110144290874e-05, 1.0]]
iteration: 30
inliers: 
[0, 68, 69, 76, 82, 83, 85, 87, 94, 95, 280, 294, 301, 308, 310, 316, 319, 320, 324, 325, 363, 364, 365, 366, 369, 374, 376, 378, 381, 382, 383, 384, 392, 395, 396]
best inliers: 
[28, 44, 50, 55, 56, 57, 66, 67, 68, 69, 76, 82, 83, 85, 87, 98, 11

inliers: 
[89, 230, 310, 394]
best inliers: 
[111, 115, 120, 122, 125, 129, 133, 134, 136, 139, 140, 141, 142, 143, 144, 145, 146, 161, 164, 166, 167, 187, 190, 206, 214, 232, 233, 234, 235, 236, 240, 243, 247, 248, 252, 254, 255, 256, 257, 258, 263, 272, 273, 274, 275, 276, 277, 280, 282, 284, 285, 287, 292, 293, 294, 298, 299, 303, 308, 310, 311, 312, 316, 319, 320, 323, 324, 325, 328, 330, 334, 353, 354, 363, 364, 365, 366, 367, 369, 372, 376, 378, 383, 384, 385, 387, 388, 390, 393, 394, 401]
new random indices: 
[296, 332, 381, 401]
Computed homography matrix: 
[[1.086701683903463, 0.001815821173882204, 1087.0851115417179], [-0.04421636757901511, 1.0177999373140547, -8.80194741934512], [-4.7783366862903645e-06, -8.55309980275455e-06, 1.0]]
iteration: 46
inliers: 
[296, 297, 331, 332, 333, 336, 337, 338, 366, 381, 396, 401]
best inliers: 
[111, 115, 120, 122, 125, 129, 133, 134, 136, 139, 140, 141, 142, 143, 144, 145, 146, 161, 164, 166, 167, 187, 190, 206, 214, 232, 233, 234, 235, 

inliers: 
[20, 162, 278, 376]
best inliers: 
[111, 115, 120, 122, 125, 129, 133, 134, 136, 139, 140, 141, 142, 143, 144, 145, 146, 161, 164, 166, 167, 187, 190, 206, 214, 232, 233, 234, 235, 236, 240, 243, 247, 248, 252, 254, 255, 256, 257, 258, 263, 272, 273, 274, 275, 276, 277, 280, 282, 284, 285, 287, 292, 293, 294, 298, 299, 303, 308, 310, 311, 312, 316, 319, 320, 323, 324, 325, 328, 330, 334, 353, 354, 363, 364, 365, 366, 367, 369, 372, 376, 378, 383, 384, 385, 387, 388, 390, 393, 394, 401]
new random indices: 
[77, 219, 290, 313]
Computed homography matrix: 
[[-0.39747294882184614, -0.10026590438943069, 1764.1179298486238], [-0.3968599928615184, 0.30152892527046693, 681.5137322094095], [-0.00036300038351926104, -6.03646201332552e-05, 1.0]]
iteration: 61
inliers: 
[77, 219, 290, 291, 313]
best inliers: 
[111, 115, 120, 122, 125, 129, 133, 134, 136, 139, 140, 141, 142, 143, 144, 145, 146, 161, 164, 166, 167, 187, 190, 206, 214, 232, 233, 234, 235, 236, 240, 243, 247, 248, 252, 254,

inliers: 
[35, 36, 37, 43, 46, 51, 52, 53, 54, 63, 66, 67, 187, 190, 200, 205, 206, 331, 340, 345, 350, 360, 362, 368, 428, 429, 430, 432]
best inliers: 
[111, 115, 120, 122, 125, 129, 133, 134, 136, 139, 140, 141, 142, 143, 144, 145, 146, 161, 164, 166, 167, 187, 190, 206, 214, 232, 233, 234, 235, 236, 240, 243, 247, 248, 252, 254, 255, 256, 257, 258, 263, 272, 273, 274, 275, 276, 277, 280, 282, 284, 285, 287, 292, 293, 294, 298, 299, 303, 308, 310, 311, 312, 316, 319, 320, 323, 324, 325, 328, 330, 334, 353, 354, 363, 364, 365, 366, 367, 369, 372, 376, 378, 383, 384, 385, 387, 388, 390, 393, 394, 401]
new random indices: 
[3, 55, 114, 357]
Computed homography matrix: 
[[0.7006365714915714, 0.030271300367719512, 1370.135309763673], [-0.12287863276384665, 0.9036068654804892, 124.35120049266521], [-8.186449912362205e-05, 3.776682556714177e-06, 1.0]]
iteration: 77
inliers: 
[1, 3, 4, 6, 8, 11, 14, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 39, 44, 55, 56, 68, 69, 

...

inliers: 
[104, 176, 177, 179, 181, 183, 185, 186, 188, 189, 191, 192, 195, 196, 199, 204, 207, 208, 212, 216, 218, 221, 222, 223, 224, 226, 227, 228, 229, 231, 237, 238, 240, 244, 260, 277, 280, 294, 296, 297, 301, 308, 310, 312, 316, 317, 319, 320, 324, 325, 328, 330, 334, 361, 363, 364, 365, 366, 367, 369, 370, 374, 376, 378, 381, 382, 392, 415, 434, 435]
best inliers: 
[110, 111, 114, 115, 116, 117, 120, 122, 125, 129, 131, 133, 134, 136, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 152, 157, 158, 161, 164, 167, 170, 176, 177, 178, 179, 180, 181, 183, 185, 186, 188, 189, 191, 192, 195, 196, 199, 202, 204, 207, 208, 212, 216, 218, 221, 222, 223, 224, 226, 227, 228, 229, 231, 232, 233, 234, 235, 236, 237, 243, 247, 248, 252, 254, 255, 256, 257, 258, 263, 267, 268, 272, 273, 274, 275, 276, 277, 279, 280, 282, 284, 285, 287, 289, 290, 291, 292, 296, 297, 300, 302, 304, 307, 310, 311, 312, 319, 320, 323, 324, 328, 329, 330, 334, 336, 341, 342, 361, 363, 364, 365, 367, 369, 374, 376

[119, 133, 253, 343]
Computed homography matrix: 
[[0.637424865340434, -0.0009097512465345402, 1419.756649032531], [-0.12303700581157719, 0.8429868836376969, 162.42127663223548], [-8.043107389521391e-05, -1.8606622197666342e-05, 1.0]]
iteration: 952
inliers: 
[112, 115, 116, 118, 119, 120, 122, 126, 127, 133, 134, 136, 139, 140, 141, 142, 143, 144, 145, 146, 166, 167, 239, 253, 259, 269, 343, 356, 398, 402]
best inliers: 
[110, 111, 114, 115, 116, 117, 120, 122, 125, 129, 131, 133, 134, 136, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 152, 157, 158, 161, 164, 167, 170, 176, 177, 178, 179, 180, 181, 183, 185, 186, 188, 189, 191, 192, 195, 196, 199, 202, 204, 207, 208, 212, 216, 218, 221, 222, 223, 224, 226, 227, 228, 229, 231, 232, 233, 234, 235, 236, 237, 243, 247, 248, 252, 254, 255, 256, 257, 258, 263, 267, 268, 272, 273, 274, 275, 276, 277, 279, 280, 282, 284, 285, 287, 289, 290, 291, 292, 296, 297, 300, 302, 304, 307, 310, 311, 312, 319, 320, 323, 324, 328, 329, 330, 334, 336

inliers: 
[23, 24, 25, 26, 27, 153, 154, 202, 232, 233, 234, 235, 236, 237, 243, 247, 248, 252, 254, 255, 257, 294]
best inliers: 
[110, 111, 114, 115, 116, 117, 120, 122, 125, 129, 131, 133, 134, 136, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 152, 157, 158, 161, 164, 167, 170, 176, 177, 178, 179, 180, 181, 183, 185, 186, 188, 189, 191, 192, 195, 196, 199, 202, 204, 207, 208, 212, 216, 218, 221, 222, 223, 224, 226, 227, 228, 229, 231, 232, 233, 234, 235, 236, 237, 243, 247, 248, 252, 254, 255, 256, 257, 258, 263, 267, 268, 272, 273, 274, 275, 276, 277, 279, 280, 282, 284, 285, 287, 289, 290, 291, 292, 296, 297, 300, 302, 304, 307, 310, 311, 312, 319, 320, 323, 324, 328, 329, 330, 334, 336, 341, 342, 361, 363, 364, 365, 367, 369, 374, 376, 378, 382, 392, 395, 428, 429, 430, 432]
new random indices: 
[5, 27, 59, 192]
Computed homography matrix: 
[[5.167135593071812, -0.9542036783514725, 1230.618374808748], [2.2926084979725383, 0.4533651207067403, -19.296942057812114], [0.00163117

inliers: 
[11, 14, 16, 19, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 39, 64, 189, 191, 199, 204, 209, 210, 215, 246, 250, 251, 309, 335, 343, 344, 346, 347, 348, 349, 352, 353, 354, 355, 356, 357, 358, 359, 372, 379, 380, 398, 399, 402, 406, 408, 409, 410, 414, 419]
best inliers: 
[110, 111, 114, 115, 116, 117, 120, 122, 125, 129, 131, 133, 134, 136, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 152, 157, 158, 161, 164, 167, 170, 176, 177, 178, 179, 180, 181, 183, 185, 186, 188, 189, 191, 192, 195, 196, 199, 202, 204, 207, 208, 212, 216, 218, 221, 222, 223, 224, 226, 227, 228, 229, 231, 232, 233, 234, 235, 236, 237, 243, 247, 248, 252, 254, 255, 256, 257, 258, 263, 267, 268, 272, 273, 274, 275, 276, 277, 279, 280, 282, 284, 285, 287, 289, 290, 291, 292, 296, 297, 300, 302, 304, 307, 310, 311, 312, 319, 320, 323, 324, 328, 329, 330, 334, 336, 341, 342, 361, 363, 364, 365, 367, 369, 374, 376, 378, 382, 392, 395, 428, 429, 430, 432]
new random indices: 
[216, 235, 292, 392]


'\nfor i in range(0,len(mp1)):\n    error = find_error(mp1[i],mp2[i],H)\n    print(error)\n'

In [57]:
result = stitch_func(img1,img2,H)
cv2.imwrite("./results/resultant.jpg",result)

2592 3888
2592 3888


True

In [58]:
print(np.shape(img3))
print(np.shape(result))
#raise NotImplementedError
mp1,mp2 = detect_sift(result,img3)

(2592, 3888)
(3296, 5958)
Matching key points wait for 1-2 minutes
Ratio:0.0
Matched keypoints for0, 0
(4.403958320617676, 1790.380859375) (4.403958320617676, 1337.380859375)
Ratio:0.07696258111511667
Matched keypoints for1, 1
(23.343111038208008, 948.1729125976562) (23.24932289123535, 495.0573425292969)
Ratio:0.0
Matched keypoints for2, 2
(36.191375732421875, 1798.939453125) (36.191375732421875, 1345.939453125)
Ratio:0.0
Matched keypoints for3, 3
(39.51446533203125, 1799.0230712890625) (39.51446533203125, 1346.0230712890625)
Ratio:0.13704491279096734
Matched keypoints for4, 4
(39.93378448486328, 665.256103515625) (40.092952728271484, 211.48292541503906)
Ratio:0.0
Matched keypoints for5, 5
(44.21464157104492, 1799.4029541015625) (44.21464157104492, 1346.4029541015625)
Ratio:0.10854991957985655
Matched keypoints for6, 7
(59.538543701171875, 939.1791381835938) (59.67233657836914, 486.35186767578125)
Ratio:0.3376231319074679
Matched keypoints for7, 8
(60.97882843017578, 637.995849609375) 

Ratio:0.0
Matched keypoints for76, 83
(401.39599609375, 1834.9483642578125) (401.39599609375, 1381.9483642578125)
Ratio:0.10783506724098443
Matched keypoints for77, 84
(409.74468994140625, 1280.7034912109375) (409.7431335449219, 827.7381591796875)
Ratio:0.32774338306917994
Matched keypoints for79, 85
(423.7689208984375, 1862.763671875) (423.8370056152344, 1409.67431640625)
Ratio:0.09260795315930714
Matched keypoints for80, 86
(428.6385498046875, 947.841552734375) (428.6575622558594, 494.80145263671875)
Ratio:0.10828250837413049
Matched keypoints for81, 87
(437.9768981933594, 918.1221313476562) (438.2464599609375, 465.0484313964844)
Ratio:0.0
Matched keypoints for82, 88
(444.46368408203125, 1870.005615234375) (444.46368408203125, 1417.005615234375)
Ratio:0.0
Matched keypoints for83, 89
(451.1660461425781, 1870.57666015625) (451.1660461425781, 1417.57666015625)
Ratio:0.0
Matched keypoints for84, 90
(459.1259765625, 1862.3367919921875) (459.1259765625, 1409.3367919921875)
Ratio:0.0
Matche

Ratio:0.2629206292421186
Matched keypoints for149, 158
(885.8796997070312, 1087.115966796875) (885.8631591796875, 634.2350463867188)
Ratio:0.17695656596044843
Matched keypoints for150, 159
(889.0216064453125, 1584.1871337890625) (889.2686157226562, 1131.2791748046875)
Ratio:0.0
Matched keypoints for151, 160
(891.8342895507812, 1064.3873291015625) (891.8342895507812, 611.3873291015625)
Ratio:0.1488196397056843
Matched keypoints for152, 163
(907.33154296875, 915.5467529296875) (907.351318359375, 462.6355285644531)
Ratio:0.0
Matched keypoints for153, 164
(914.6802368164062, 1075.96533203125) (914.6802368164062, 622.96533203125)
Ratio:0.0
Matched keypoints for154, 165
(915.9512329101562, 1846.4664306640625) (915.9512329101562, 1393.4664306640625)
Ratio:0.0
Matched keypoints for155, 166
(924.4087524414062, 1843.763671875) (924.4087524414062, 1390.763671875)
Ratio:0.13024868369481993
Matched keypoints for156, 167
(927.913330078125, 1233.4124755859375) (928.5992431640625, 780.4559936523438)
R

Ratio:0.0
Matched keypoints for221, 233
(1245.35400390625, 2198.1591796875) (1245.35400390625, 1745.1590576171875)
Ratio:0.0
Matched keypoints for222, 234
(1245.35400390625, 2198.1591796875) (1245.35400390625, 1745.1590576171875)
Ratio:0.1980717789103209
Matched keypoints for223, 235
(1247.88623046875, 860.791015625) (1247.8004150390625, 407.8173522949219)
Ratio:0.2682391157831256
Matched keypoints for224, 239
(1248.049072265625, 2211.2783203125) (1248.347900390625, 1758.2926025390625)
Ratio:0.0
Matched keypoints for225, 236
(1248.1029052734375, 2212.40283203125) (1248.1029052734375, 1759.40283203125)
Ratio:0.0
Matched keypoints for226, 237
(1248.1029052734375, 2212.40283203125) (1248.1029052734375, 1759.40283203125)
Ratio:0.0
Matched keypoints for227, 238
(1248.2093505859375, 1859.1693115234375) (1248.2093505859375, 1406.1693115234375)
Ratio:0.0
Matched keypoints for228, 240
(1252.2740478515625, 2205.144775390625) (1252.2740478515625, 1752.1448974609375)
Ratio:0.0
Matched keypoints fo

Ratio:0.0
Matched keypoints for300, 310
(1505.3612060546875, 1878.671875) (1505.3612060546875, 1425.671875)
Ratio:0.0
Matched keypoints for301, 311
(1506.9295654296875, 1885.7164306640625) (1506.9295654296875, 1432.7164306640625)
Ratio:0.0
Matched keypoints for302, 312
(1507.1695556640625, 2755.52294921875) (1507.1695556640625, 2302.52294921875)
Ratio:0.0
Matched keypoints for303, 313
(1508.201904296875, 1904.4007568359375) (1508.201904296875, 1451.4007568359375)
Ratio:0.0
Matched keypoints for304, 314
(1513.40087890625, 1889.0872802734375) (1513.40087890625, 1436.0872802734375)
Ratio:0.0
Matched keypoints for305, 315
(1517.70458984375, 2724.3408203125) (1517.70458984375, 2271.3408203125)
Ratio:0.0
Matched keypoints for306, 316
(1518.8551025390625, 1904.499267578125) (1518.8551025390625, 1451.499267578125)
Ratio:0.0
Matched keypoints for307, 317
(1524.3487548828125, 1904.7310791015625) (1524.3487548828125, 1451.7310791015625)
Ratio:0.0
Matched keypoints for308, 319
(1529.5135498046875,

...

Ratio:0.0
Matched keypoints for1194, 1152
(3198.3291015625, 2417.8076171875) (3198.3291015625, 1964.8076171875)
Ratio:0.0
Matched keypoints for1195, 1153
(3200.105712890625, 1516.3438720703125) (3200.105712890625, 1063.3438720703125)
Ratio:0.0
Matched keypoints for1196, 1155
(3202.43798828125, 1644.2188720703125) (3202.43798828125, 1191.2188720703125)
Ratio:0.0
Matched keypoints for1197, 1156
(3202.66259765625, 1644.3951416015625) (3202.66259765625, 1191.3951416015625)
Ratio:0.0
Matched keypoints for1199, 1158
(3205.810546875, 1511.97314453125) (3205.810546875, 1058.97314453125)
Ratio:0.19941324386933224
Matched keypoints for1200, 1159
(3206.55078125, 1522.9146728515625) (3206.776123046875, 1070.125)
Ratio:0.6040429164260347
Matched keypoints for1201, 1159
(3206.812744140625, 1523.9864501953125) (3206.776123046875, 1070.125)
Ratio:0.15830462190827915
Matched keypoints for1202, 1160
(3207.177001953125, 1047.6182861328125) (3207.13623046875, 595.1687622070312)
Ratio:0.0
Matched keypoints

Ratio:0.0
Matched keypoints for1288, 1241
(3353.792724609375, 2483.59716796875) (3353.792724609375, 2030.59716796875)
Ratio:0.0
Matched keypoints for1289, 1242
(3353.792724609375, 2483.59716796875) (3353.792724609375, 2030.59716796875)
Ratio:0.0
Matched keypoints for1290, 1243
(3353.792724609375, 2483.59716796875) (3353.792724609375, 2030.59716796875)
Ratio:0.0
Matched keypoints for1291, 1244
(3355.531005859375, 2502.62744140625) (3355.531005859375, 2049.62744140625)
Ratio:0.0
Matched keypoints for1292, 1245
(3357.721435546875, 2329.5439453125) (3357.721435546875, 1876.5439453125)
Ratio:0.0
Matched keypoints for1293, 1246
(3358.802490234375, 3012.286376953125) (3358.802490234375, 2559.286376953125)
Ratio:0.0
Matched keypoints for1294, 1247
(3359.123291015625, 2466.5146484375) (3359.123291015625, 2013.5146484375)
Ratio:0.0
Matched keypoints for1295, 1248
(3359.123291015625, 2466.5146484375) (3359.123291015625, 2013.5146484375)
Ratio:0.0
Matched keypoints for1296, 1249
(3359.123291015625

Ratio:0.0
Matched keypoints for1368, 1315
(3445.054443359375, 1040.2454833984375) (3445.054443359375, 587.2454833984375)
Ratio:0.41258759807244805
Matched keypoints for1369, 1316
(3447.560302734375, 1527.450439453125) (3447.4814453125, 1074.316650390625)
Ratio:0.24929173720834708
Matched keypoints for1370, 1317
(3451.025390625, 1306.8328857421875) (3450.7548828125, 853.743896484375)
Ratio:0.0
Matched keypoints for1371, 1318
(3451.1494140625, 1914.9727783203125) (3451.1494140625, 1461.9727783203125)
Ratio:0.0
Matched keypoints for1372, 1319
(3451.74951171875, 1039.9678955078125) (3451.74951171875, 586.9678955078125)
Ratio:0.0
Matched keypoints for1373, 1320
(3451.74951171875, 1039.9678955078125) (3451.74951171875, 586.9678955078125)
Ratio:0.0
Matched keypoints for1374, 1321
(3453.806640625, 2516.569580078125) (3453.806640625, 2063.569580078125)
Ratio:0.0
Matched keypoints for1375, 1322
(3453.865966796875, 1045.605712890625) (3453.865966796875, 592.6056518554688)
Ratio:0.0
Matched keypoi

Ratio:0.0
Matched keypoints for1453, 1402
(3572.802001953125, 2668.9453125) (3572.802001953125, 2215.9453125)
Ratio:0.0
Matched keypoints for1454, 1403
(3577.144775390625, 2877.249267578125) (3577.144775390625, 2424.249267578125)
Ratio:0.0
Matched keypoints for1455, 1404
(3580.057861328125, 2496.843994140625) (3580.057861328125, 2043.843994140625)
Ratio:0.0
Matched keypoints for1456, 1405
(3580.365966796875, 2783.476806640625) (3580.365966796875, 2330.476806640625)
Ratio:0.0
Matched keypoints for1457, 1406
(3582.879638671875, 2837.46875) (3582.879638671875, 2384.46875)
Ratio:0.0
Matched keypoints for1458, 1407
(3583.026123046875, 2443.92919921875) (3583.026123046875, 1990.9290771484375)
Ratio:0.0
Matched keypoints for1459, 1408
(3583.026123046875, 2443.92919921875) (3583.026123046875, 1990.9290771484375)
Ratio:0.0
Matched keypoints for1460, 1409
(3584.213134765625, 2828.983642578125) (3584.213134765625, 2375.983642578125)
Ratio:0.0
Matched keypoints for1461, 1410
(3587.027099609375, 10

Ratio:0.0
Matched keypoints for1543, 1486
(3790.113037109375, 891.2249755859375) (3790.113037109375, 438.2249755859375)
Ratio:0.0
Matched keypoints for1544, 1487
(3790.113037109375, 891.2249755859375) (3790.113037109375, 438.2249755859375)
Ratio:0.0
Matched keypoints for1546, 1488
(3795.041748046875, 1560.404296875) (3795.041748046875, 1107.404296875)
Ratio:0.0
Matched keypoints for1550, 1490
(3798.46533203125, 897.102294921875) (3798.46533203125, 444.1023254394531)
Ratio:0.10689512553118696
Matched keypoints for1554, 1491
(3813.431396484375, 923.539306640625) (3813.333984375, 470.55584716796875)
Ratio:0.3852344263245949
Matched keypoints for1555, 1492
(3816.66650390625, 896.150634765625) (3816.9580078125, 443.19525146484375)
Ratio:0.0
Matched keypoints for1556, 1493
(3817.087158203125, 2951.450927734375) (3817.087158203125, 2498.450927734375)
Ratio:0.0
Matched keypoints for1557, 1494
(3817.087158203125, 2951.450927734375) (3817.087158203125, 2498.450927734375)
Ratio:0.0
Matched keypoi

In [59]:
print(mp1[0])
print(mp2[0])

(4.403958320617676, 1790.380859375)
(4.403958320617676, 1337.380859375)


In [60]:
H,inliers = ransac_func(mp1,mp2)
#inliers.sort()
print("final inliers")
print(inliers)

[131, 1168, 1239, 1373]
Computed homography matrix: 
[[1.0026970162434452, 0.01219122754419266, -13.712615357614743], [-0.0006167808689238178, 1.0114251010710225, -463.9507233338009], [-4.0814444962368634e-07, 3.617845183618513e-06, 1.0]]
iteration: 0
inliers: 
[9, 18, 24, 27, 34, 43, 66, 69, 84, 85, 94, 96, 103, 104, 109, 119, 125, 128, 129, 131, 142, 182, 193, 263, 354, 367, 392, 394, 397, 408, 416, 444, 473, 504, 544, 556, 562, 563, 568, 569, 575, 582, 604, 629, 653, 715, 718, 738, 739, 741, 746, 748, 749, 769, 772, 819, 822, 828, 829, 830, 831, 832, 833, 834, 835, 841, 843, 846, 847, 848, 850, 854, 855, 856, 857, 858, 859, 862, 866, 867, 871, 872, 874, 875, 877, 879, 885, 886, 887, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901, 904, 905, 906, 907, 910, 911, 912, 913, 914, 915, 918, 919, 920, 921, 922, 923, 924, 925, 927, 930, 931, 932, 933, 936, 937, 938, 939, 941, 942, 943, 944, 945, 949, 951, 952, 954, 955, 957, 958, 959, 960, 961, 964, 965, 966, 967, 968, 969, 972, 

In [61]:
f_result = stitch_func(result,img3,H)
cv2.imwrite("./results/final_resultant.jpg",result)

3296 5958
2592 3888


True